# PUSHPENDRA

In [1]:
import requests as rq
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.alert import Alert

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}


In [3]:
Short=["DC","SRH","PKBS","MI","CSK","KKR","RR","RCB","LSG","GT"]
type_match=["League (N)","Final (N)","Qualifier 2 (N)","Eliminator (N)","Qualifier 1 (N)"]
# url[1]

In [4]:
url=["https://www.espncricinfo.com/series/ipl-2021-1249214/match-schedule-fixtures-and-results","https://www.espncricinfo.com/series/indian-premier-league-2022-1298423/match-schedule-fixtures-and-results","https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/match-schedule-fixtures-and-results","https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/match-schedule-fixtures-and-results"]

In [13]:
match_list=[]
base_url="https://www.espncricinfo.com/"
for u in url:
    res=rq.get(u,headers=headers)
    # print(res.status_code)
    soup =bs(res.text,'html.parser')
    anchor=soup.find_all("a",class_="ds-inline-flex ds-items-start ds-leading-none")
    for a  in anchor:
        link=a.get("href")
        if link and "live-cricket-score"in link:
            match_list.append(base_url+link)
len(match_list)         

282

In [8]:
match_data = {
        'Match Type':[],
        'POTM': [],
        'Venue': [],
        'Team1': [],
        'Team2': [],
        'Toss_Winner':[],
        'Toss_Decision': [],
        'Winner': [],
        'Win_Margin':[],
        'Result_Type': [],
        'Target_Runs': [],
         'Match':[],
         'Season':[]
}


In [ ]:

match_data = {
    "Match Type": [],
    "Result_Type": [],
    "POTM": [],
    "Venue": [],
    "Team1": [],
    "Team2": [],
    "Toss_Winner": [],
    "Toss_Decision": [],
    "Winner": [],
    "Win_Margin": [],
    "Target_Runs": [],
    "Match": [],
    "Season": []
}

# Define type_match and Short if not already defined
type_match = ["Qualifier 1 (N)", "Eliminator (N)", "Qualifier 2 (N)", "Final (N)"]
Short = ["KKR", "RCB", "MI", "CSK", "SRH", "RR", "DC", "PBKS"]

j = 1
ses = 2021
for match in match_list:
    mat_res = rq.get(match, headers=headers)
    soup = bs(mat_res.text, 'html.parser')
    team = soup.find_all("span", class_="ds-text-tight-l ds-font-bold ds-text-typo hover:ds-text-typo-primary ds-block ds-truncate")
    team = [t.text.strip() for t in team]
    temp = soup.find("div", class_="ds-text-tight-m ds-font-regular ds-text-typo-mid3")
    temp = temp.text.strip().split(sep=",")
    venue = temp[1]
    match_type = temp[0]
    if match_type not in type_match:
        match_type = "League (N)"
    Runs_t = soup.find_all("div", class_="ds-py-2 ds-px-3 ds-border-b ds-border-line ds-uppercase ds-flex ds-justify-between")
    if len(Runs_t) < 2:
        continue 
    target = int(Runs_t[0].find_all("strong")[1].text.strip().split('/')[0].split()[1]) + 1
    wicket_2 = int(Runs_t[1].find_all("strong")[1].text.strip().split('/')[1])
    T2_run = int(Runs_t[1].find_all("strong")[1].text.strip().split('/')[0].split()[1])
    Result = soup.find("p", class_="ds-text-tight-s ds-font-medium ds-truncate ds-text-typo").text.strip()
    if T2_run >= target:
        winner = team[1]
        win_margin = f"{10 - wicket_2} Wickets"
        result_type = "Wickets"
    elif T2_run < target - 1:
        winner = team[0]
        win_margin = f"{target - 1 - T2_run} Runs"
        result_type = "Runs"
    else:
        for i in Result:
            if i in Short:
                winner = i
                break
        result_type = "Draw"
        win_margin = 0
    potm = soup.find("span", class_="ds-text-tight-m ds-font-medium ds-text-typo ds-underline ds-decoration-ui-stroke hover:ds-text-typo-primary hover:ds-decoration-ui-stroke-primary ds-block ds-cursor-pointer").text.strip()
    toss = soup.find("span", class_="ds-text-tight-s ds-font-regular").text.strip()
    for i in range(2):
        if team[i] in toss:
            toss_winner = team[i]
        if "field" in toss:
            toss_decision = "Field_First"
        else:
            toss_decision = "Bat_First"
    match_data["Match Type"].append(match_type)
    match_data["Result_Type"].append(result_type)
    match_data["POTM"].append(potm)
    match_data["Venue"].append(venue)
    match_data["Team1"].append(team[0])
    match_data["Team2"].append(team[1])
    match_data["Toss_Winner"].append(toss_winner)
    match_data["Toss_Decision"].append(toss_decision)
    match_data["Winner"].append(winner)
    match_data["Win_Margin"].append(win_margin)
    match_data["Target_Runs"].append(target)
    match_data["Match"].append(j)
    match_data["Season"].append(ses)
    j += 1
    if match_type == "Final (N)":
        j = 1
        ses += 1

# Convert match_data to DataFrame
data = pd.DataFrame(match_data)

print(f"Total matches in DataFrame: {len(data)}")

Total matches found: 282
Total matches in DataFrame: 278


In [14]:
pd.set_option('display.max_rows', None) 
pd.reset_option('display.max_rows') 
len(data) 

278

In [11]:
data=pd.DataFrame(match_data)

In [12]:
data.to_csv("Season_2021-2024.csv")